In [16]:
import hashlib
import time

In [5]:
class Hash256(object):
    
    def __init__(self, string):
        self.string = string
    
    
    def packBits(self):
        binLis = [bin(ord(i))[2:] for i in self.string]
        binLis = ['0'*(8-len(i))+i for i in binLis]
        return binLis
    
    
    def alignBits(self):
        binLis = self.packBits()
        chunk = []
        if len(binLis) > 448/8:
            for i in range(len(binLis)//(448//8)+1):
                chunk += [binLis[i*448//8:(i+1)*448//8]]
        else:
            chunk += [binLis]
        chunk[-1] = chunk[-1] + ['00000000']*((448//8)-len(chunk[-1]))
        
        return chunk
    
    
    def fullBits(self):
        chunk = self.alignBits()
        binary = bin(448)[2:]
        last = (512//8-len(binary))*'0'+binary
        last = '1'+last[1:]
        
        for i in range(len(chunk)-1):
            chunk[i] = chunk[i] + [last]
            
        try:
            ind = chunk[-1].index('00000000')
            chunk[-1][ind] = '10000000'
            binary = bin(ind*8)[2:]
            last = (512//8-len(binary))*'0'+binary
            chunk[-1] = chunk[-1] + [last]
        except:
            chunk[-1] = chunk[-1] + [last]
        
        return chunk
    
            
    def initChunk(self):
        chunk = self.fullBits()
        allChunks = []
        for i in range(len(chunk)):
            chunk[i] = "".join(chunk[i])
            tmp = []
            for o in range(512//32):
                tmp += [chunk[i][o*32:(o+1)*32]]
            allChunks += [tmp]
            
        return allChunks
    
    
    def xor(self,x,y):
        out = bin(int(x,2) ^ int(y,2))[2:]
        return '0'*(32-len(out))+out
    
    def or_(self,x,y):
        out = bin(int(x,2) | int(y,2))[2:]
        return '0'*(32-len(out))+out
    
    def and_(self,x,y):
        out = bin(int(x,2) & int(y,2))[2:]
        return '0'*(32-len(out))+out
    
    def Lshift(self,x):
        mask = 2 ** 32 -1
        out = (int(x,2) << 1) & mask
        out = bin(out)[2:]
        out = '0'*(32-len(out))+out
        return out
    
    def not_(self,x):
        out = bin(~int(x,2))
        if out[0] == '-':
            out = out[3:]
        else:
            out = out[2:]
        return '0'*(32-len(out))+out
    
    def binSum(self,x,y):
        out = ''
        for i in range(len(x)):
            if int(x[i]) == 1 and x[i] == y[i]:
                out += '1'
            elif int(x[i]) == 0 and x[i] == y[i]:
                out += '0'
            elif x[i] != y[i]:
                out += '1'
        return out
        
    
    def primeHash(self):
        allChunks = self.initChunk()
        for chunk in range(len(allChunks)):
            for i in range(16,80):
                wordA = allChunks[chunk][i-3]
                wordB = allChunks[chunk][i-8]
                wordC = allChunks[chunk][i-14]
                wordD = allChunks[chunk][i-16]
                
                xorA = self.xor(wordA,wordB)
                xorB = self.xor(xorA,wordC)
                xorC = self.xor(xorB,wordD)
                
                xorC = self.Lshift(xorC)
                
                allChunks[chunk] += [xorC]
                
        return allChunks
    
    
    def secondHash(self):
        allChunks = self.primeHash()      
        k = '11111101100110001100000011010010'
        h0 = k
        h1 = k
        h2 = k
        h3 = k
        h4 = k
        h5 = k
        h6 = k
        h7 = k
        
        for chunk in allChunks:
            for row in range(80):
                if row < 20:
                    t = self.xor(k,chunk[row])
                    t = self.binSum(t,chunk[row])
                    t = self.not_(t)
                    if row < 10:
                        h0 = self.xor(h0,t)
                    else:
                        h1 = self.or_(h1,t)
                elif row < 40:
                    t = self.and_(k,chunk[row])
                    t = self.binSum(t,chunk[row])
                    t = self.not_(t)
                    if row < 30:
                        h2 = self.xor(h2,t)
                    else:
                        h3 = self.or_(h3,t)
                elif row < 60:
                    t = self.or_(k,chunk[row])
                    t = self.binSum(t,chunk[row])
                    t = self.not_(t)
                    if row < 50:
                        h4 = self.xor(h4,t)
                    else:
                        h5 = self.or_(h5,t)
                else:
                    t = self.Lshift(chunk[row])
                    t = self.binSum(t,chunk[row])
                    t = self.not_(t)
                    if row < 70:
                        h6 = self.xor(h6,t)
                    else:
                        h7 = self.or_(h7,t)
                        
            h0 = self.and_(h1,h2)
            h1 = self.and_(h2,h3)
            h2 = self.and_(h3,h4)
            h3 = self.and_(h4,h5)
            h4 = self.and_(h5,h6)
            h5 = self.and_(h6,h7)
            h6 = self.and_(h7,h0)
            h7 = self.and_(h0,h1)
        
        h0 = hex(int('1'+h0[1:],2))[2:]
        h1 = hex(int('1'+h1[1:],2))[2:]
        h2 = hex(int('1'+h2[1:],2))[2:]
        h3 = hex(int('1'+h3[1:],2))[2:]
        h4 = hex(int('1'+h4[1:],2))[2:]
        h5 = hex(int('1'+h5[1:],2))[2:]
        h6 = hex(int('1'+h6[1:],2))[2:]
        h7 = hex(int('1'+h7[1:],2))[2:]
        
        return h0+h1+h2+h3+h4+h5+h6+h7


In [34]:
text = open('hash.txt','r').read()
print("Text len: ",len(text))

t = time.time()
hashlib.sha224(text.encode()).hexdigest()
d0 = time.time()-t

t = time.time()
print(Hash256(text).secondHash())
d1 = time.time()-t

print('Mine is this much slower: ',d1/d0)

Text len:  25327
d36081d4d36181d08d4fb3208d4fb3208bc402408bc40248d36081c0d36081d0
Mine is this much slower:  1348.5782097649187
